In [44]:
import osmnx as ox
import pandas as pd
import numpy as np
import random
from shapely.geometry import Point
import geopandas as gpd
import json
import concurrent.futures
import time
import geopandas as gpd

In [45]:
df = pd.read_csv('Zip_zori_uc_sfrcondomfr_sm_month.csv', dtype={'RegionName':'str'})
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2023-11-30,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1608.145192,...,2031.526482,2017.766107,2006.276682,2026.201026,2039.152261,2064.044270,2067.249444,2082.408869,2086.030434,2096.459479
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1322.999718,...,1875.237067,1883.844334,1897.292953,1909.784089,1919.376336,1929.856600,1924.841529,1925.223470,1937.729818,1949.967285
2,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2322.417849,2282.500000
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1249.562365,...,1825.754710,1830.712458,1814.003417,1820.641420,1822.235419,1846.918002,1859.795531,1866.252869,1859.417399,1855.383088
4,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1460.857466,1478.183156,1479.396040,1479.012474,1471.360057,1469.007981,1484.654672,1501.266583,1514.329707,1504.006109


In [46]:
df = df.dropna(subset='City')

In [47]:
df = df[['RegionID','SizeRank','RegionName','RegionType','StateName','State','City','Metro','CountyName','2024-08-31']].dropna(subset='2024-08-31')
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2024-08-31
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,2096.459479
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1949.967285
2,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2282.500000
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1855.383088
4,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,1504.006109


In [5]:
with open('geospatial_data/amenity_cities.json', 'r') as f:
    city_metadata = json.load(f)

In [108]:
def fetch_amenities(city):
    amenities = ox.geometries_from_place(city, tags={'amenity': True})
    print(f"Finished {city}")
    return amenities

sampled_df = df.sample(n=100)

unique_cities = (sampled_df['City'] + ', ' + sampled_df['State']).unique()

timeout_threshold = 20

with concurrent.futures.ThreadPoolExecutor() as executor:
    for city in list(unique_cities):
        if city in city_metadata or type(city) != str:
            continue
        print(f"Attempting {city}")
        future = executor.submit(fetch_amenities, city)
        try:
            amenities = future.result(timeout=timeout_threshold)
            amenities = amenities[amenities.geometry.type == 'Point']
            amenities['longitude'] = amenities.geometry.x
            amenities['latitude'] = amenities.geometry.y
            df_trimmed = amenities.reset_index()[['amenity', 'longitude', 'latitude']]
            df_trimmed.to_csv(f"geospatial_data/{city}.csv")
            city_metadata.append(city)
        except concurrent.futures.TimeoutError:
            break

Attempting East Riverdale, MD


C:\Users\Jeffrey\AppData\Local\Temp\ipykernel_14872\397918523.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  amenities = ox.geometries_from_place(city, tags={'amenity': True})


Finished East Riverdale, MD


In [112]:
with open('geospatial_data/amenity_cities.json', 'w') as f:
    json.dump(city_metadata, f, indent=4)

In [48]:
with open('geospatial_data/amenity_cities.json', 'r') as f:
    city_metadata = json.load(f)

In [49]:
zipcode_df = pd.read_csv('geospatial_data/uszips.csv', dtype={'zip':'str'})
zipcode_df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16834.0,100.9,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37642.0,479.2,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,49075.0,551.7,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,00606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5590.0,48.7,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25542.0,265.7,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [50]:
zip_dict = {row['zip']: (row['lat'], row['lng']) for _, row in zipcode_df.iterrows()}

In [51]:
from math import radians, sin, cos, sqrt, atan2

In [52]:
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def calculate_amenities_within_radius(df):
    amenity_types = ['restaurant', 'bank','fast_food','cafe','place_of_worship','school','bicycle_rental']
    for amenity in amenity_types:
        df[f'num_{amenity}'] = 0
    df['has_features'] = True
    # Iterate over each row (ZIP code) in the DataFrame
    for idx, row in df.iterrows():
        zip_code = str(row['RegionName'])
        city = row['City'] + ', ' + row['State']
        
        if city not in city_metadata:
            df.at[idx, 'has_features'] = False
            continue
        if zip_code not in zip_dict:
            df.at[idx, 'has_features'] = False
            continue

        zip_amenities = pd.read_csv(f"geospatial_data/{city}.csv")
        zip_center_lat = zip_dict[zip_code][0]
        zip_center_lon = zip_dict[zip_code][1]
        # Count the number of each amenity type within the specified radius (2 miles)
        for amenity in amenity_types:
            filtered_amenities = zip_amenities[zip_amenities['amenity'] == amenity]
            count = 0

            # Calculate the distance of each amenity to the ZIP code center
            for _, amenity_row in filtered_amenities.iterrows():
                amenity_lat = amenity_row['latitude']
                amenity_lon = amenity_row['longitude']
                distance = haversine(zip_center_lat, zip_center_lon, amenity_lat, amenity_lon)
                if distance <= 50:
                    count += 1

            # Update the DataFrame with the count of amenities
            df.at[idx, f'num_{amenity}'] = count

    return df

In [53]:
nearby_amenities = calculate_amenities_within_radius(df)
nearby_amenities.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2024-08-31,num_restaurant,num_bank,num_fast_food,num_cafe,num_place_of_worship,num_school,num_bicycle_rental,has_features
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,2096.459479,0,0,0,0,0,0,0,False
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1949.967285,0,0,0,0,0,0,0,False
2,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2282.500000,0,0,0,0,0,0,0,False
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1855.383088,817,118,684,183,930,76,41,True
4,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,1504.006109,0,0,0,0,0,0,0,False


In [58]:
nearby_amenities = nearby_amenities[nearby_amenities['has_features']]

In [59]:
nearby_amenities.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2024-08-31,num_restaurant,num_bank,num_fast_food,num_cafe,num_place_of_worship,num_school,num_bicycle_rental,has_features
3,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,1855.383088,817,118,684,183,930,76,41,True
8,95992,10,90011,zip,CA,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,2514.250000,2034,191,1158,615,713,343,126,True
10,84630,12,60629,zip,IL,IL,Chicago,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,1244.958333,1999,204,903,711,933,295,865,True
12,96361,14,91331,zip,CA,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,3800.000000,2034,191,1158,615,713,343,126,True
13,74101,15,37013,zip,TN,TN,Nashville,"Nashville-Davidson--Murfreesboro--Franklin, TN",Davidson County,1738.937689,283,27,173,68,759,193,11,True


In [62]:
nearby_amenities.max()

RegionID                                                      418163
SizeRank                                                       30158
RegionName                                                     99224
RegionType                                                       zip
StateName                                                         WY
State                                                             WY
City                                                           Yukon
Metro                   Washington-Arlington-Alexandria, DC-VA-MD-WV
CountyName                                               York County
2024-08-31                                                 11449.375
num_restaurant                                                  2034
num_bank                                                         204
num_fast_food                                                   1158
num_cafe                                                         722
num_place_of_worship              

In [61]:
nearby_amenities.to_csv('nearby_amenities.csv', index=False)

In [70]:
import folium

df = pd.read_csv('geospatial_data/Irvine, CA.csv')

amenity_colors = {
    'restaurant': 'red',
    'bank': 'blue',
    'fast_food': 'green',
    'cafe': 'orange',
    'place_of_worship': 'purple',
    'school': 'yellow',
    'bicycle_rental': 'black'
}

map_center = [df['latitude'].mean(), df['longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=14)

for _, row in df.iterrows():
    if row['amenity'] not in amenity_colors:
        continue
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=8,
        color=amenity_colors[row['amenity']],
        fill=True,
        fill_color=amenity_colors[row['amenity']],
        fill_opacity=0.6,
        popup=row['amenity']
    ).add_to(mymap)

legend_html = '''
<div style="position: fixed;
     bottom: 50px; left: 50px; width: 150px; height: 120px;
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     ">
     <b>&nbsp; Amenity Legend</b><br>
     &nbsp; <i class="fa fa-circle" style="color:red"></i>&nbsp; Restaurant<br>
     &nbsp; <i class="fa fa-circle" style="color:blue"></i>&nbsp; Cafe<br>
     &nbsp; <i class="fa fa-circle" style="color:green"></i>&nbsp; Bank<br>
     &nbsp; <i class="fa fa-circle" style="color:orange"></i>&nbsp; Park<br>
     &nbsp; <i class="fa fa-circle" style="color:purple"></i>&nbsp; School<br>
</div>
'''

mymap.get_root().html.add_child(folium.Element(legend_html))

mymap.save('amenities_map_with_legend.html')

In [71]:
mymap